<a href="https://colab.research.google.com/github/lazybuttrying/TensorFlow-Certificate/blob/main/C4/W2/ungraded_labs/C4_W2_Lab_1_features_and_labels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

**Note:** This notebook can run using TensorFlow 2.5.0

시계열 데이터는

feature로 window size를, 즉 30일 단위로 끊는 것

label로 그 윈도우의 마지막이자 다음번째의 첫번째 값을 사용

In [1]:
!pip install tensorflow==2.6.2

     |████████████████████████████████| 458.3 MB 2.6 kB/s 
     |████████████████████████████████| 462 kB 44.0 MB/s 
     |████████████████████████████████| 1.3 MB 45.1 MB/s 
     |████████████████████████████████| 5.6 MB 43.6 MB/s 
  Created wheel for clang: filename=clang-5.0-py3-none-any.whl size=30692 sha256=56c38080758133908828e2e8562a49c01c85d1ca87d15b1d6bb601caaa0a9d4a
  Stored in directory: /root/.cache/pip/wheels/98/91/04/971b4c587cf47ae952b108949b46926f426c02832d120a082a
  Created wheel for wrapt: filename=wrapt-1.12.1-cp37-cp37m-linux_x86_64.whl size=68721 sha256=05dfc7ff1cf433b97ebba3b26d747f238cc5c689c32f216b682b3498ef13f329
  Stored in directory: /root/.cache/pip/wheels/62/76/4c/aa25851149f3f6d9785f6c869387ad82b3fd37582fa8147ac6
Successfully built clang wrapt
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.2
    Uninstalling typing-extensions-3.10.0.2:
      Successfully uninstalled typing-extensions-3.10.0.2
  Attemptin

In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
print(tf.__version__)

2.6.2


In [3]:
dataset = tf.data.Dataset.range(10)
for val in dataset:
   print(val.numpy())

0
1
2
3
4
5
6
7
8
9


In [16]:
dataset = tf.data.Dataset.range(10)
# 5칸씩 잡아가면서 봄. 다 끝나면 한 칸씩 이동
# 끝까지 보려고 하기에 윈도우 사이즈보다 작은 데이터도 다룸
dataset = dataset.window(5, shift=1)
for window_dataset in dataset:
  print("\n->", end=" ")
  for val in window_dataset:
    print(val.numpy(), end=" ")



-> 0 1 2 3 4 
-> 1 2 3 4 5 
-> 2 3 4 5 6 
-> 3 4 5 6 7 
-> 4 5 6 7 8 
-> 5 6 7 8 9 
-> 6 7 8 9 
-> 7 8 9 
-> 8 9 
-> 9 

In [22]:
dataset = tf.data.Dataset.range(10)
# drop_remainder : truncate remainder 
# 딱 그 때의 window에 담기는 것만 추출
dataset = dataset.window(5, shift=1, drop_remainder=True)
for window_dataset in dataset:
  print("\n->", end=" ")
  for val in window_dataset:
    print(val.numpy(), end=" ")


-> 0 1 2 3 4 
-> 1 2 3 4 5 
-> 2 3 4 5 6 
-> 3 4 5 6 7 
-> 4 5 6 7 8 
-> 5 6 7 8 9 

In [20]:
dataset = tf.data.Dataset.range(10)
# winow.numpy()로 리스트 안에 감싸기
# 그래서 10개의 데이터를 5칸씩 1 step 단위로 보니 총 6번.
dataset = dataset.window(5, shift=1, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(5))
for window in dataset:
  print("\n->", end=" ")
  print(window.numpy(), end="")



-> [0 1 2 3 4]
-> [1 2 3 4 5]
-> [2 3 4 5 6]
-> [3 4 5 6 7]
-> [4 5 6 7 8]
-> [5 6 7 8 9]

In [19]:
dataset = tf.data.Dataset.range(10)
dataset = dataset.window(5, shift=1, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(5))
dataset = dataset.map(lambda window: (window[:-1], window[-1:]))
# dataset = dataset.shuffle(buffer_size=10) # 각 행의 순서를 섞음

# 마지막 하나가 바로 next value이기에 label y로 설정
# 나머지는 window 크기를 4로 생각하여 x
for x,y in dataset:
  print("\n->", end=" ")
  print(x.numpy(), y.numpy(), end="")


-> [0 1 2 3] [4]
-> [1 2 3 4] [5]
-> [2 3 4 5] [6]
-> [3 4 5 6] [7]
-> [4 5 6 7] [8]
-> [5 6 7 8] [9]

In [9]:
dataset = tf.data.Dataset.range(10)
dataset = dataset.window(5, shift=1, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(5))
dataset = dataset.map(lambda window: (window[:-1], window[-1:]))
dataset = dataset.shuffle(buffer_size=10)
dataset = dataset.batch(2).prefetch(1) # 총 6개의 x를 각 2개의 크기를 가진 소집단으로 분리
# 그래서 총 3개의 x와 y가 생성됨
for x,y in dataset:
  print("x = ", x.numpy())
  print("y = ", y.numpy())

x =  [[2 3 4 5]
 [3 4 5 6]]
y =  [[6]
 [7]]
x =  [[1 2 3 4]
 [0 1 2 3]]
y =  [[5]
 [4]]
x =  [[4 5 6 7]
 [5 6 7 8]]
y =  [[8]
 [9]]
